In [2]:
# TASK GENERATOR
import torch
from torch.utils.data import Dataset
import numpy as np
from scipy.ndimage import gaussian_filter1d

class PainTaskGenerator(Dataset):
    def __init__(self, task_dict):
        # INPUT DIMENSIONS
        self.stimulus_condition = task_dict['stimulus_condition'] # stimulus intensity
        self.onset_offset_noise  = task_dict['onset_offset_noise'] # mean and std
        self.stimulus_intensities = task_dict['stimulus_intensities'] 
        # TIME SETTINGS
        self.epoch_onsets = task_dict['epoch_onsets']
        self.epoch_durations = task_dict['epoch_durations']
        self.trial_length = task_dict['trial_length']
        assert len(self.epoch_onsets) == len(self.epoch_durations)
        # ETCs...
        self.pain_temperature = task_dict['pain_temperature']
        self.warmth_temperature = task_dict['warmth_temperature']

    def __getitem__(self, idx):
        # ITEMS TO RETURN
        n_epochs  = len(self.epoch_onsets)
        input_seq = np.zeros((3, self.trial_length))
        output_seq = np.zeros((3, self.trial_length))
        pain_condition = self.stimulus_condition

        # 1. input_seq. 3 dim. 
        # dim 1 for continous stimulus intensities.
        # dim 2 for warmth indicator 
        # dim 3 for pain indicator (above 46)
        # e.g. for dim 2 & 3
        # [0 0 1 1 1 1 1 1 1 1 0 0;
        #  0 0 0 0 1 1 1 0 0 0 0 0]
        for i in range(n_epochs):
            fromm       = self.epoch_onsets[i] 
            tilll       = fromm + self.epoch_durations[i]
            onset_stim  = self.stimulus_intensities[i][0]
            offset_stim = self.stimulus_intensities[i][1]
            input_seq[0, fromm:tilll] = np.linspace(onset_stim, offset_stim, num=tilll-fromm) 
        input_seq[1, :] = input_seq[0, :] >= self.warmth_temperature
        input_seq[2, :] = input_seq[0, :] >= self.pain_temperature

        # 2. output_seq. 3 dim.
        # dim 1 for continuous ratings. (requires behavioral model first...?)
        # dim 2 for not pain indicator [pain as 1, non-pain as 0] with noise added.
        # dim 3 for 1 - dim2
        
        output_seq[0, :] = gaussian_filter1d(np.roll(input_seq[0, :], 2), sigma=1) 
        # THIS NEEDS MODIFICATION. "np.convolve"... gaussian_filter1d takes zero mean gaussian distribution which is not likely the case of the "lagged" behavioral consequences.
        
        onset_offset_noise = self.onset_offset_noise
        pain_onset = np.rint(np.random.normal(onset_offset_noise[0][0], onset_offset_noise[0][1], 1)).astype(int)[0]
        pain_offset = np.rint(np.random.normal(onset_offset_noise[1][0], onset_offset_noise[1][1], 1)).astype(int)[0]

        output_seq[1, :pain_onset] = 1
        output_seq[1, pain_onset:pain_offset] = 0
        output_seq[1, pain_offset:] = 1
        output_seq[2, :] = 1 - output_seq[1, :]


        return {'input_seq': input_seq, 'output_seq': output_seq,
                'pain_onoffset': (pain_onset, pain_offset), 'condition': pain_condition}
        
        # For output dim 1 convolution 

In [3]:
# Make Task Params
stim_ints   = [43, 44, 45, 46.5, 47, 47.5]
conditions  = ["141", "151", "161", "242", "252", "262", "343", "353", "363"]
stimulus_intensities = []
for cond_i in range(len(conditions)):
    start_end_temp = stim_ints[int(conditions[cond_i][0]) - 1]
    middle_temp = stim_ints[int(conditions[cond_i][1]) - 1]
    stimulus_intensities.append([[25, start_end_temp], [start_end_temp, start_end_temp], 
                                   [start_end_temp, middle_temp], [middle_temp, middle_temp], 
                                   [middle_temp, start_end_temp], [start_end_temp, start_end_temp],
                                   [start_end_temp, 25]])
onoffset_noise = [(14, 1), [23, 1]]

# Check
int_cond = 0     
task_params = {'stimulus_condition': conditions[int_cond],
                'stimulus_intensities': stimulus_intensities[int_cond],
                'onset_offset_noise': onoffset_noise,
                'epoch_onsets':    [0, 2, 8, 17, 22, 31, 37],
                'epoch_durations': [2, 6, 9, 5, 9, 6, 2],
                'trial_length': 39, # 0 - 39 sec
                'pain_temperature': 46,
                'warmth_temperature': 40} 
data = PainTaskGenerator(task_params)

In [5]:
# RNN class define
import torch
import torch.nn as nn
import torch.optim as optim
device = 'mps'

class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNN, self).__init__() # 1. WHAT IS THIS?
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.rnn_layer = nn.RNN(input_dim, hidden_dim, nonlinearity="relu", dropout=1/8, batch_first=True) # batch_first = True => Input's first dim is the batch size
                                                                         # 3rd input num_layer which means vertically extended layer.
        self.fc_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x): # What is X? ... model(input) == model.forward(input). input => # batch x time x hidden_dim
        hidden = torch.zeros(x.size(0), x.size(1), self.hidden_dim) 
        output = torch.zeros(x.size(0), x.size(1), self.output_dim)
        
        h_t    = torch.zeros(x.size(0), self.hidden_dim).to(device)
        for t in range(x.size(1)): # iteration for time
                                   # this part depends on how precise we want to model the neural activity.
                                   # this is just for the computation. just using vanila RNN
            o_t, h_t = self.rnn_layer(x[:, t, :])
            hidden[:, t, :] = h_t
            output[:, t, :] = o_t
        return output, hidden
    
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dt):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.i2h = nn.Linear(input_dim, hidden_dim)
        self.h2h = nn.Linear(hidden_dim, hidden_dim)
        self.h2o = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        hidden = torch.zeros(x.size(0), x.size(1), self.hidden_dim)
        output = torch.zeros(x.size(0), x.size(1), self.output_dim)

        h = torch.zeros(x.size(0), self.hidden_dim).to(device)
        for t in range(x.size(1)):
            h = h * (1 - self.dt/self.tau) + (self.dt/self.tau) * torch.relu(self.i2h(x[:,t,:]) + self.h2h(h))
            o = self.h2o(h)
            hidden[:,t,:] = h
            output[:,t,:] = o
        return output, hidden
        

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

batch_size = 18
num_epochs = 3000
learning_rate = 1e-2
trial_length = 39
input_dim  = 3
output_dim  = 3
hidden_dim = 8
device = 'mps'
model = RNN(input_dim, hidden_dim, output_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_history = []

for epoch in range(num_epochs):
    inputs = torch.zeros((batch_size, trial_length, input_dim))
    targets = torch.zeros((batch_size, trial_length, output_dim))
    for i in range(batch_size):
        int_cond = i % 9
        task_params = {'stimulus_condition': conditions[int_cond],
                'stimulus_intensities': stimulus_intensities[int_cond],
                'onset_offset_noise': onoffset_noise,
                'epoch_onsets':    [0, 2, 8, 17, 22, 31, 37],
                'epoch_durations': [2, 6, 9, 5, 9, 6, 2],
                'trial_length': 39, # 0 - 39 sec
                'pain_temperature': 46,
                'warmth_temperature': 40} # This part + condition indexing can be incorporated in the "PainTaskGenerator"
        single_cond       = PainTaskGenerator(task_params)
        single_cond_trial = single_cond[0]
        inputs[i], targets[i] = torch.tensor(single_cond_trial['input_seq'].T), torch.tensor(single_cond_trial['input_seq'].T)

    inputs.to(device)
    targets.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    # outputs.grad()
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())

    if epoch % 100 == 0:
        print (f'Training epoch ({epoch+1}/{num_epochs}), Loss: {loss.item():3.3f}')


TypeError: RNN.__init__() missing 1 required positional argument: 'dt'

In [6]:
test_batch_size = 512
Button_threshold = 0.9
dataset = PainTaskGenerator(task_params)
with torch.no_grad():
    inputs = torch.zeros((test_batch_size, trial_length, input_dim))
    targets = torch.zeros((test_batch_size, trial_length, output_dim))
    for i in range(test_batch_size):
        data = dataset[i]
        inputs, targets = torch.tensor(data['input_seq']).T, torch.tensor(data['output_seq']).T
    targets

In [7]:
from sklearn.decomposition import PCA
test_batch_size = 512

dataset = PainTaskGenerator(task_params)
with torch.no_grad():
    inputs = torch.zeros((test_batch_size, trial_length, input_dim))
    targets = torch.zeros((test_batch_size, trial_length, output_dim))
    for i in range(test_batch_size):
        data = dataset[i]
        inputs[i], targets[i] = torch.tensor(data['input_seq']).T, torch.tensor(data['output_seq']).T
    inputs.to(device)

    outputs, hidden_state = model(inputs)
    outputs = outputs.cpu().numpy()
    hidden_state = hidden_state.cpu().numpy()
    targets = targets.cpu().numpy()

pca_model = PCA(n_components=3)
pca_model.fit(hidden_state.reshape(-1,hidden_dim))
reduced_hidden_state = np.zeros((test_batch_size, trial_steps, 3))
for i in range(test_batch_size):
    reduced_hidden_state[i] = pca_model.transform(hidden_state[i])

NameError: name 'VisualDiscrimination' is not defined

In [8]:
task_params

{'stimulus_condition': '363',
 'stimulus_intensities': [[25, 45],
  [45, 45],
  [45, 47.5],
  [47.5, 47.5],
  [47.5, 45],
  [45, 45],
  [45, 25]],
 'onset_offset_noise': [(14, 1), [23, 1]],
 'epoch_onsets': [0, 2, 8, 17, 22, 31, 37],
 'epoch_durations': [2, 6, 9, 5, 9, 6, 2],
 'trial_length': 39,
 'pain_temperature': 46,
 'warmth_temperature': 40}